# Install detectron2

In [1]:
!pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.
# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 KB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=8a0863349e970bb8fb4fa30c15bb002f9194fc731da48d4e53f70a6af2985cd1
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 21.10.1 requires cupy-cuda114, which is not installed.
pytorch-lightning 1.6.0 requires PyYAML>=5.4, but you have pyyaml 5.1 which is incompatible.
kubernetes 23.3.0 requires pyyaml>=5.4.1, but you have pyyaml 5.1 which is incompatible.
dask 2022.2.0 

In [2]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [3]:
dataset_path = "../input/pavement-image-dataset-coco"

# Train on a custom dataset

In [4]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("crack_train", {}, dataset_path + "/train/_annotations.coco.json", dataset_path + "/train")
register_coco_instances("crack_val", {}, dataset_path + "/valid/_annotations.coco.json", dataset_path + "/valid")


## Train!



In [5]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("crack_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 50000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 12  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[04/18 04:18:05 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f6e8b1.pkl: 243MB [00:03, 74.1MB/s]                           


[04/18 04:18:14 d2.engine.train_loop]: Starting training from iteration 0
[04/18 04:18:27 d2.utils.events]:  eta: 4:11:22  iter: 19  total_loss: 4.983  loss_cls: 2.586  loss_box_reg: 0.07107  loss_rpn_cls: 2.18  loss_rpn_loc: 0.1481  time: 0.3042  data_time: 0.0147  lr: 4.9953e-06  max_mem: 2548M
[04/18 04:18:33 d2.utils.events]:  eta: 4:12:37  iter: 39  total_loss: 4.087  loss_cls: 2.359  loss_box_reg: 0.08928  loss_rpn_cls: 1.542  loss_rpn_loc: 0.09888  time: 0.3037  data_time: 0.0055  lr: 9.9902e-06  max_mem: 2548M
[04/18 04:18:39 d2.utils.events]:  eta: 4:15:45  iter: 59  total_loss: 3.301  loss_cls: 1.941  loss_box_reg: 0.1553  loss_rpn_cls: 1.124  loss_rpn_loc: 0.1059  time: 0.3085  data_time: 0.0059  lr: 1.4985e-05  max_mem: 2548M
[04/18 04:18:46 d2.utils.events]:  eta: 4:15:55  iter: 79  total_loss: 2.764  loss_cls: 1.429  loss_box_reg: 0.1812  loss_rpn_cls: 1.004  loss_rpn_loc: 0.1255  time: 0.3099  data_time: 0.0070  lr: 1.998e-05  max_mem: 2548M
[04/18 04:18:52 d2.utils.even

In [6]:
# # Look at training curves in tensorboard:
# %load_ext tensorboard
# %tensorboard --logdir output

## Inference & evaluation using the trained model
Now, let's run inference with the trained model on the balloon validation dataset. First, let's create a predictor using the model we just trained:



In [7]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

Then, we randomly select several samples to visualize the prediction results.

We can also evaluate its performance using AP metric implemented in COCO API.
This gives an AP of ~70. Not bad!

In [8]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("crack_val", ("bbox",), False, output_dir="./output")
val_loader = build_detection_test_loader(cfg, "crack_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`

WARNING [04/18 08:41:30 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/18 08:41:30 d2.data.datasets.coco]: Loaded 1336 images in COCO format from ../input/pavement-image-dataset-coco/valid/_annotations.coco.json
[04/18 08:41:31 d2.data.build]: Distribution of instances among all 12 categories:
|   category    | #instances   |   category    | #instances   |   category    | #instances   |
|:-------------:|:-------------|:-------------:|:-------------|:-------------:|:-------------|
|    Cracks     | 0            | Aligator Cr.. | 863          |  Block Crack  | 1736         |
| Lane Longit.. | 2204         | Longitudina.. | 546          | Reflective .. | 2384         |
| Sealed Bloc.. | 0            | Sealed Long.. | 2534         | Sealed Refl.. | 1073         |
| Sealed Tran.. | 0            | Transvers C.. | 942          |    pothole    | 124          |
|               |              |               |              | 

In [9]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
cocoGt = COCO(dataset_path + "/valid/_annotations.coco.json")
cocoDt = cocoGt.loadRes("output/coco_instances_results.json")
cocoEval = COCOeval(cocoGt,cocoDt,"bbox")
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.05s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.35s).
Accumulating evaluation results...
DONE (t=0.52s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.114
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.327
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.048
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.092
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.158
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.087
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.180
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

In [10]:
# from detectron2.evaluation import COCOEvaluator, inference_on_dataset
# from detectron2.data import build_detection_test_loader
# from detectron2.config import get_cfg
# from detectron2 import model_zoo
# from detectron2.engine import DefaultPredictor
# from detectron2.utils.logger import setup_logger
# from detectron2.data import MetadataCatalog

# setup_logger()
# cfg_c4 = get_cfg()
# cfg_c4.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_C4_1x.yaml"))
# cfg_c4.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_C4_1x.yaml")
# predictor_c4 = DefaultPredictor(cfg_c4)

# evaluator_c4 = COCOEvaluator("coco_2017_val", ("bbox",), False, output_dir="./output")
# val_loader_c4 = build_detection_test_loader(cfg_c4, "coco_2017_val")
# print(inference_on_dataset(predictor_c4.model, val_loader_c4, evaluator_c4))